In [1]:
## Import lib
import torch
import torch.nn.functional as F
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
import scipy.io as sio
import numpy as np

In [2]:
## CNN Creation
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes = 10):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
## Define hyperparameters
input_size = 625 * 90 ## We have 90 subcarriers and 625 points for each subcarrier
num_classes = 2
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [5]:
## Create DataLoader for data
class Dataset(DataLoader):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).int()

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [6]:
## Load data and create dataset
_CSI = []
_y = []

for file in ['home_lab(L)', 'home_lab(R)', 'lecture_room', 'living_room', 'meeting_room']:
    _mat = sio.loadmat(f'data/dataset_{file}.mat')
    _CSI.append(_mat.get('dataset_CSI'))
    _y.append(_mat.get('dataset_labels'))

X = np.concatenate(_CSI)
y = np.concatenate(_y)

## Shuffle the dataset
rows = [_ for _ in range(X.shape[0])]
np.random.shuffle(rows)

X = X[rows]
y = y[rows]

## Split the dataset into two sub datasets (ont for the train and one for the test)
rows_train_percentage = .85
rows_train = int(rows_train_percentage * X.shape[0])

X_train = X[:rows_train, :, :]
y_train = y[:rows_train, :]

X_test = X[rows_train:, :, :]
y_test = y[rows_train:, :]

train_loader: Dataset = Dataset(X = X_train, y = y_train)
test_loader: Dataset = Dataset(X = X_test, y = y_test)

In [7]:
## Initialize model
model = CNN(in_channels = 64, num_classes = num_classes).to(device)

In [8]:
## Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
## Train the network
for epoch in range(num_epochs):
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    
    ## Loop through batches
    for _b in range(int(len(train_loader) / batch_size)):
        data, targets = train_loader[_b*batch_size:(_b+1)*batch_size]

        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        print(f'Loss : {loss}')

        optimizer.step()

Epoch [1/10]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x3432 and 784x2)